# smooth and fit GLM

Loop over subject and TR, thereby concatenating across session

In [1]:
from bids.layout import BIDSLayout
from nipype.interfaces import afni 
from nipype.interfaces.io import BIDSDataGrabber, DataFinder, DataSink, DataGrabber
import nipype.pipeline as pe
import nipype as ni
from nipype.interfaces.utility import Function
import nipype.interfaces.fsl.maths as fsl
from nipype.interfaces import spm as spm
from nipype.algorithms import modelgen as mgen
from nipype.algorithms.misc import Gunzip 
import pandas as pd
import os, re, json
# https://nipype.readthedocs.io/en/0.11.0/users/spmmcr.html

matlab_cmd = '/opt/spm12-r7219/run_spm12.sh /opt/matlabmcr-2010a/v713/ script'
spm.SPMCommand.set_mlab_paths(matlab_cmd=matlab_cmd, use_mcr=True)

In [2]:
Basedir = "/scratch/qbi/uqkgarn1/STRIWP1/"
layout = BIDSLayout(Basedir)
subs = layout.get_subjects()

glm = pe.Workflow(name="glms") # workflow to run the analysis

/opt/miniconda-latest/envs/neuro/lib/python3.6/site-packages/bids/layout/models.py:102: FutureWarning: The 'extension' entity currently excludes the leading dot ('.'). As of version 0.14.0, it will include the leading dot. To suppress this warning and include the leading dot, use `bids.config.set_option('extension_initial_dot', True)`.
  FutureWarning)


## Data grabbing and saving nodes

Note: I copied the motion files from the derivatives/etc path to derivatives/glm/sub-etc and I removed the T2 echo-1 and echo-2 .nii.gz data from the func folder below. I also removed the echo 2 motion files from the same folder, and the bjson files (renaming echo 1 of the 700 TR to fit the below field template

In [3]:
dgT2s = pe.Node(DataGrabber(infields=['sub', 'TR'], 
                            outfields=['func','motion','onsets','bjson','mask']), name='T2-grabber')
dgT2s.inputs.base_dir = "/scratch/qbi/uqkgarn1/STRIWP1/"
dgT2s.inputs.sort_filelist = True
dgT2s.inputs.template='*'
dgT2s.inputs.template_args = {'func': [['sub', 'sub', 'TR']],
                              'motion':[['sub', 'sub', 'TR']],
                              'onsets':[['sub', 'sub', 'TR']],
                              'bjson':[['sub', 'sub', 'TR']],
                              'mask':[['sub', 'sub', 'TR']]}
dgT2s.inputs.field_template = {'func': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-%s/ses-*/func/sub-%s_*TR%s_space-T1w_desc-preproc_bold.nii.gz',
                               'motion': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-%s/ses-*/func/sub-%s_*-TR%s*desc-motion_regressors.txt',
                               'onsets': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-%s/ses-*/func/sub-%s_*-TR%s_glm_onsets.json',
                               'bjson': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-%s/ses-*/func/sub-%s_*-TR%s*space-T1w_desc-preproc_bold.json',
                               'mask': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-%s/ses-02/func/sub-%s_*TR%s_space-T1w_desc-brain_mask.nii.gz'}


In [4]:
# # debugging
dgT2s.inputs.sub = '04'
dgT2s.inputs.TR = '700'
res = dgT2s.run()
res.outputs

210522-10:38:18,214 nipype.workflow INFO:
	 [Node] Setting-up "T2-grabber" in "/tmp/tmpfui84zc9/T2-grabber".
210522-10:38:18,222 nipype.workflow INFO:
	 [Node] Running "T2-grabber" ("nipype.interfaces.io.DataGrabber")
210522-10:38:18,234 nipype.workflow INFO:
	 [Node] Finished "T2-grabber".



bjson = ['/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-04/ses-02/func/sub-04_ses-02_task-learnAtt_acq-TR700_echo-1_space-T1w_desc-preproc_bold.json', '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-04/ses-02/func/sub-04_ses-02_task-learnAtt_acq-TR700_echo-2_space-T1w_desc-preproc_bold.json', '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-04/ses-02/func/sub-04_ses-02_task-learnAtt_acq-TR700_space-T1w_desc-preproc_bold.json', '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-04/ses-03/func/sub-04_ses-03_task-learnAtt_acq-TR700_echo-1_space-T1w_desc-preproc_bold.json', '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-04/ses-03/func/sub-04_ses-03_task-learnAtt_acq-TR700_echo-2_space-T1w_desc-preproc_bold.json', '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-04/ses-03/func/sub-04_ses-03_task-learnAtt_acq-TR700_space-T1w_desc-preproc_bold.json', '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-04/ses-04/func/sub-04_ses-04_task-learnAtt_acq-TR700_echo-1_space-T1w_desc-preproc_bold.json', '/scratch/

In [5]:
T2inf = pe.Node(ni.IdentityInterface(fields=['sub', 'TR']),
                   name='T2-info')
T2inf.iterables = [('sub', ['01', '02', '03', '04', '05']), ('TR', ['700', '1510', '1920'])]
#T2inf.iterables = [('sub', ['01', '02', '03']), ('TR', ['700', '1510', '1920'])]
#T2inf.iterables = [('sub', ['04']), ('TR', ['700', '1510', '1920'])]
T2inf.iterables 

[('sub', ['01', '02', '03', '04', '05']), ('TR', ['700', '1510', '1920'])]

In [6]:
def printSubPath(fullFilePath):
    # function to split filepath into constituent parts, then print string to add as input to DataSink for the container string
    # given the full filepath, this extracts the subject folder and TR strings for input
    # into DataSink
    import os
    import re
    fname = os.path.normpath(fullFilePath[0])
    fname
    l = fname.split(os.sep)
    TR = re.search('.*acq-TR([0-9]*)_.*',l[-1])
    TR = str(int(TR.group(1)))
    name = [s for s in l if re.search('sub', s)][0]
    name = [name, "TR"+TR]
    name
    name = '/'.join(name)
    return name

# Data sink


In [7]:
ds = pe.Node(DataSink(), name='sink-stuff')
ds.inputs.base_directory = "/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/"
# glm Time and Dispersion derivatives and FAST correction of serial correlations
substitutions = [('_TR_([0-9]*)_sub_([0-9]*)', '')]
ds.inputs.regexp_substitutions = substitutions

# Get design info

In [8]:
def select_regressors(bunches, conditions):
    # this function allows you to isolate the regressors that match the list of conditions, 
    # and returns them in the prt format for spm12
    from nipype.interfaces.base import Bunch
    nu_output = []
    count = 0
    
    for bunch in bunches:
        count = count + 1
        idx = [i for i,c in enumerate(bunch.conditions) if c in conditions]
        nuonsets = []
        nudurs = []
        for i in idx:
            nuonsets.append(bunch.onsets[i])
            nudurs.append([0])
            
        nu_output.insert(count, 
                         Bunch(conditions=conditions,
                               onsets=nuonsets,
                               durations=nudurs))
    return nu_output

In [9]:
selectOnsets = pe.Node(Function(input_names=['bunches', 'conditions'],
                                output_names=['nu_output'],
                                function=select_regressors),
                        name='selectOnsets')
selectOnsets.inputs.conditions = ['att_left_5', 'att_left_8', 'att_right_5', 'att_right_8']
#selectOnsets.inputs.conditions = ['left_hand', 'right_hand']

In [10]:
# the input function should be a list of the prt files, taken across sessions, for each TR
def getOnsetsJson(input_files):
    from nipype.interfaces.base import Bunch
    import json
    prt_output = [] #prt=protocol
    count = 0
    for f in input_files: 
        count = count + 1
        with open(f, "r+") as file:
            data = json.load(file)
            prt_output.insert(count, 
                              Bunch(conditions=data['names'],
                                    onsets=data['onsets'],
                                    durations=data['durations']))
    return prt_output

In [11]:
getOnsets = pe.Node(Function(input_names=['input_files'],
                             output_names=['prt_output'],
                             function=getOnsetsJson),
                    name='get_prt_onsets')

# Gunzip Nodes

1 for functional data, 1 for mask (condense during finesse)

In [12]:
gunzipfunc = pe.MapNode(Gunzip(), name='gunzipfunc', iterfield=['in_file'])

In [13]:
gmask = pe.Node(Gunzip(), name='m-zip')

# Smooth the functional data

In [14]:
smooth = pe.Node(spm.Smooth(), name='smoooooth')
smooth.inputs.fwhm = [2, 2, 2]

# Get the TR for the model spec

In [15]:
# the input function should be a list of the prt files, taken across sessions, for each TR
def getTRJson(input_files):
    import json
    with open(input_files[0], "r+") as file:
            data = json.load(file)
            TR = data['RepetitionTime'] 
    if TR < .5:
        TR = 1.92
    return TR

In [16]:
getTR = pe.Node(Function(input_names=['input_files'],
                         output_names=['TR'],
                         function=getTRJson),
                name='get_TR')

In [17]:
def printSavFol(TR):
    # function to append the TR to a name for the save folder
    name = "TR" + TR
    return name

# Specify GLM Model

In [18]:
# SpecifyModel - Generates SPM-specific Model
modelspec = pe.Node(mgen.SpecifySPMModel(concatenate_runs=False,
                                         input_units='secs',
                                         output_units='secs',
                                         high_pass_filter_cutoff=128),
                    name="modelspec")

# Generate design matrix

In [19]:
# Level1Design - Generates an SPM design matrix
level1design = pe.Node(spm.Level1Design(bases={'hrf': {'derivs': [1, 1]}},
                                 timing_units='secs',
                                 model_serial_correlations='FAST'),
                       name="level1design")

# Estimate model

In [20]:
# EstimateModel - estimate the parameters of the model
estimate = pe.Node(spm.EstimateModel(estimation_method={'Classical': 1},
                                     write_residuals=False),
                                     name="estimate")

# inputs
# spm mat file
# outputs
# beta_images
# residual_images
# spm_mat_file

# will input the spm.mat files output by the design module above

# # EstimateContrast - estimates contrasts
# level1conest = pe.Node(spm.EstimateContrast(), name="level1conest")
# level1conest.inputs.contrasts = contrast_list

# Connect Workflow

In [21]:
glm.connect([(T2inf, dgT2s, [('sub',  'sub')]), 
             (T2inf, dgT2s, [('TR',   'TR')]),
             (dgT2s, ds, [(('motion', printSubPath),
                            'container')]),
             (dgT2s, getOnsets, [('onsets', 'input_files')]),
             (getOnsets, selectOnsets, [('prt_output', 'bunches')]),
             (dgT2s, gunzipfunc, [('func', 'in_file')]),
             (dgT2s, gmask, [('mask', 'in_file')]),
             (gunzipfunc, smooth, [('out_file', 'in_files')]),
             (smooth, ds, [('smoothed_files', 'DATA.@smooth')]),
             (dgT2s, getTR, [('bjson', 'input_files')]),
             (getTR, modelspec, [('TR', 'time_repetition')]),
             (dgT2s, modelspec, [('motion', 'realignment_parameters')]),
             (selectOnsets, modelspec, [('nu_output', 'subject_info')]),
             (smooth, modelspec, [('smoothed_files', 'functional_runs')]),
             (getTR, level1design, [('TR', 'interscan_interval')]),
             (modelspec, level1design, [('session_info', 'session_info')]),
             (gmask, level1design, [('out_file', 'mask_image')]),
             (level1design, estimate, [('spm_mat_file', 'spm_mat_file')]),
             (estimate, ds, [('beta_images', 'FLGLM.@beta')]),
             (estimate, ds, [('spm_mat_file', 'FLGLM.@des')]),
             (estimate, ds, [('residual_image', 'FLGLM.@res')]),
             (estimate, ds, [('mask_image', 'FLGLM.@mask')])
])
glm.run()

210522-10:38:38,695 nipype.workflow INFO:
	 Workflow glms settings: ['check', 'execution', 'logging', 'monitoring']
210522-10:38:38,915 nipype.workflow INFO:
	 Running serially.
210522-10:38:38,918 nipype.workflow INFO:
	 [Node] Setting-up "glms.T2-grabber" in "/tmp/tmpfui84zc9/T2-grabber".
210522-10:38:38,919 nipype.workflow INFO:
	 [Node] Outdated cache found for "glms.T2-grabber".
210522-10:38:38,925 nipype.workflow INFO:
	 [Node] Running "T2-grabber" ("nipype.interfaces.io.DataGrabber")
210522-10:38:38,935 nipype.workflow INFO:
	 [Node] Finished "glms.T2-grabber".
210522-10:38:38,936 nipype.workflow INFO:
	 [Node] Setting-up "glms.get_TR" in "/tmp/tmptlqs0z7a/glms/_TR_1920_sub_05/get_TR".
210522-10:38:38,941 nipype.workflow INFO:
	 [Node] Running "get_TR" ("nipype.interfaces.utility.wrappers.Function")
210522-10:38:38,945 nipype.workflow INFO:
	 [Node] Finished "glms.get_TR".
210522-10:38:38,946 nipype.workflow INFO:
	 [Node] Setting-up "glms.m-zip" in "/tmp/tmpr8ar_lck/glms/_TR_19

210522-10:50:42,154 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-05/TR1920/FLGLM/_TR_1920_sub_05/beta_0011.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-05/TR1920/FLGLM//beta_0011.nii
210522-10:50:42,170 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-05/TR1920/FLGLM/_TR_1920_sub_05/beta_0012.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-05/TR1920/FLGLM//beta_0012.nii
210522-10:50:42,187 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-05/TR1920/FLGLM/_TR_1920_sub_05/beta_0013.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-05/TR1920/FLGLM//beta_0013.nii
210522-10:50:42,204 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-05/TR1920/FLGLM/_TR_1920_sub_05/beta_0014.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-05/TR1920/FLGLM//beta_0014.nii
210522-1

210522-10:50:42,690 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-05/TR1920/FLGLM/_TR_1920_sub_05/beta_0045.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-05/TR1920/FLGLM//beta_0045.nii
210522-10:50:42,706 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-05/TR1920/FLGLM/_TR_1920_sub_05/beta_0046.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-05/TR1920/FLGLM//beta_0046.nii
210522-10:50:42,721 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-05/TR1920/FLGLM/_TR_1920_sub_05/beta_0047.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-05/TR1920/FLGLM//beta_0047.nii
210522-10:50:42,736 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-05/TR1920/FLGLM/_TR_1920_sub_05/beta_0048.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-05/TR1920/FLGLM//beta_0048.nii
210522-1

210522-10:50:43,763 nipype.workflow INFO:
	 [Node] Finished "glms.sink-stuff".
210522-10:50:43,764 nipype.workflow INFO:
	 [Node] Setting-up "glms.T2-grabber" in "/tmp/tmpfui84zc9/T2-grabber".
210522-10:50:43,765 nipype.workflow INFO:
	 [Node] Outdated cache found for "glms.T2-grabber".
210522-10:50:43,771 nipype.workflow INFO:
	 [Node] Running "T2-grabber" ("nipype.interfaces.io.DataGrabber")
210522-10:50:43,780 nipype.workflow INFO:
	 [Node] Finished "glms.T2-grabber".
210522-10:50:43,781 nipype.workflow INFO:
	 [Node] Setting-up "glms.get_TR" in "/tmp/tmp_qdly9kf/glms/_TR_1510_sub_05/get_TR".
210522-10:50:43,786 nipype.workflow INFO:
	 [Node] Running "get_TR" ("nipype.interfaces.utility.wrappers.Function")
210522-10:50:43,790 nipype.workflow INFO:
	 [Node] Finished "glms.get_TR".
210522-10:50:43,791 nipype.workflow INFO:
	 [Node] Setting-up "glms.m-zip" in "/tmp/tmpzr8j0lkh/glms/_TR_1510_sub_05/m-zip".
210522-10:50:43,796 nipype.workflow INFO:
	 [Node] Running "m-zip" ("nipype.algor

210522-11:08:23,676 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-05/TR1510/FLGLM/_TR_1510_sub_05/beta_0012.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-05/TR1510/FLGLM//beta_0012.nii
210522-11:08:23,693 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-05/TR1510/FLGLM/_TR_1510_sub_05/beta_0013.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-05/TR1510/FLGLM//beta_0013.nii
210522-11:08:23,708 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-05/TR1510/FLGLM/_TR_1510_sub_05/beta_0014.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-05/TR1510/FLGLM//beta_0014.nii
210522-11:08:23,723 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-05/TR1510/FLGLM/_TR_1510_sub_05/beta_0015.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-05/TR1510/FLGLM//beta_0015.nii
210522-1

...

210522-14:39:15,401 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR1920/FLGLM/_TR_1920_sub_01/beta_0049.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR1920/FLGLM//beta_0049.nii
210522-14:39:15,404 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR1920/FLGLM/_TR_1920_sub_01/beta_0050.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR1920/FLGLM//beta_0050.nii
210522-14:39:15,408 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR1920/FLGLM/_TR_1920_sub_01/SPM.mat -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR1920/FLGLM//SPM.mat
210522-14:39:15,459 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR1920/FLGLM/_TR_1920_sub_01/ResMS.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR1920/FLGLM//ResMS.nii
210522-14:39:15,465 nipype.i

210522-14:49:17,64 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR1510/FLGLM/_TR_1510_sub_01/beta_0010.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR1510/FLGLM//beta_0010.nii
210522-14:49:17,68 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR1510/FLGLM/_TR_1510_sub_01/beta_0011.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR1510/FLGLM//beta_0011.nii
210522-14:49:17,72 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR1510/FLGLM/_TR_1510_sub_01/beta_0012.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR1510/FLGLM//beta_0012.nii
210522-14:49:17,76 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR1510/FLGLM/_TR_1510_sub_01/beta_0013.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR1510/FLGLM//beta_0013.nii
210522-14:49

210522-14:49:17,185 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR1510/FLGLM/_TR_1510_sub_01/beta_0044.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR1510/FLGLM//beta_0044.nii
210522-14:49:17,189 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR1510/FLGLM/_TR_1510_sub_01/beta_0045.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR1510/FLGLM//beta_0045.nii
210522-14:49:17,192 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR1510/FLGLM/_TR_1510_sub_01/beta_0046.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR1510/FLGLM//beta_0046.nii
210522-14:49:17,196 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR1510/FLGLM/_TR_1510_sub_01/beta_0047.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR1510/FLGLM//beta_0047.nii
210522-1

210522-15:07:03,816 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR700/FLGLM/_TR_700_sub_01/beta_0005.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR700/FLGLM//beta_0005.nii
210522-15:07:03,818 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR700/FLGLM/_TR_700_sub_01/beta_0006.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR700/FLGLM//beta_0006.nii
210522-15:07:03,821 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR700/FLGLM/_TR_700_sub_01/beta_0007.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR700/FLGLM//beta_0007.nii
210522-15:07:03,824 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR700/FLGLM/_TR_700_sub_01/beta_0008.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR700/FLGLM//beta_0008.nii
210522-15:07:03,826 

210522-15:07:03,902 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR700/FLGLM/_TR_700_sub_01/beta_0039.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR700/FLGLM//beta_0039.nii
210522-15:07:03,905 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR700/FLGLM/_TR_700_sub_01/beta_0040.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR700/FLGLM//beta_0040.nii
210522-15:07:03,908 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR700/FLGLM/_TR_700_sub_01/beta_0041.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR700/FLGLM//beta_0041.nii
210522-15:07:03,910 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR700/FLGLM/_TR_700_sub_01/beta_0042.nii -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR700/FLGLM//beta_0042.nii
210522-15:07:03,913 